# Прогнозирование заказов такси

**Описание проекта**

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики RMSE на тестовой выборке должно быть не больше 48.

**Описание данных**

Данные лежат в файле `/datasets/taxi.csv`.

Количество заказов находится в столбце `num_orders`.

**План работы**
1. Загрузите данные и выполните их ресемплирование по одному часу.
2. Проанализируйте данные.
3. Обучите разные модели с различными гиперпараметрами. Сделайте тестовую выборку размером 10% от исходных данных.
4. Проверьте данные на тестовой выборке и сделайте выводы.

In [16]:
import pandas as pd
import numpy as np
import optuna
import plotly.express as px

from collections import defaultdict
from IPython.display import display

from ydata_profiling import ProfileReport
from fast_ml import eda
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import cross_val_predict

from sklearn.metrics import mean_squared_error

In [2]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [3]:
try:
    raw_taxi_orders = pd.read_csv('taxi.csv')
except:
    raw_taxi_orders = pd.read_csv('/datasets/taxi.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [4]:
display(eda.df_info(raw_taxi_orders))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
datetime,object,Categorical,26496,"[2018-03-01 00:00:00, 2018-03-01 00:10:00, 201...",0,0.0
num_orders,int64,Numerical,81,"[9, 14, 28, 20, 32, 21, 7, 5, 17, 12]",0,0.0


Числовые распределения:

In [5]:
display(round(raw_taxi_orders.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
num_orders,26496.0,14.07,9.21,0.0,8.0,13.0,19.0,119.0


И детальный отчет:

In [6]:
ProfileReport(raw_taxi_orders, tsmode=True).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Выводы

# Подготовка данных для ML

Данные уже достаточно чистые, поэтому обработка будет относительно быстрой:

1. Создадим новые колонки с признаками.
2. Обрежем нулевые значения.

In [7]:
df_taxi_orders = (
    raw_taxi_orders
    .loc[lambda df: df.num_orders > 0]
    .reset_index(drop=True)
    .assign(datetime=lambda df: pd.to_datetime(df.datetime))
    .set_index('datetime')
    .resample('1H').sum()
    .assign(
        hour=lambda df: df.index.hour,
        day_of_week=lambda df: df.index.dayofweek,
        is_weekend=lambda df: df.day_of_week.isin([5, 6]).astype(int),
        day_of_month=lambda df: df.index.day,
        month=lambda df: df.index.month,
        year=lambda df: df.index.year,
        lag_1=lambda df: df.num_orders.shift(1),
        mean_rol_3=lambda df: df.num_orders.rolling(window=3).mean(),
        std_rol_3=lambda df: df.num_orders.rolling(window=3).std()
    )
    .dropna()
)

print('Index is monotonically increasing:', df_taxi_orders.index.is_monotonic_increasing)
display(df_taxi_orders.head())

Index is monotonically increasing: True


,num_orders,hour,day_of_week,is_weekend,day_of_month,month,year,lag_1,mean_rol_3,std_rol_3
datetime,,,,,,,,,,
2018-03-01 02:00:00,71,2,3,0,1,3,2018,85.0,93.333333,27.465129
2018-03-01 03:00:00,66,3,3,0,1,3,2018,71.0,74.000000,9.848858
2018-03-01 04:00:00,43,4,3,0,1,3,2018,66.0,60.000000,14.933185
2018-03-01 05:00:00,6,5,3,0,1,3,2018,43.0,38.333333,30.270998
2018-03-01 06:00:00,12,6,3,0,1,3,2018,6.0,20.333333,19.857828


Теперь можем посмотреть более детальные графики: возможно, заметим некоторые зависимости в данных.

In [8]:
ProfileReport(df_taxi_orders, tsmode=True).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

И разберем на составляющие.

In [9]:
def generate_plot(y_column, title, xlim=None):
    """
    Generate a line plot for the given column with specified title and x-axis limits.

    Parameters:
    - y_column (str): The column name in df_temp to be plotted on the y-axis.
    - title (str): The title of the plot.
    - xlim (list, optional): A list of two datetime values specifying the x-axis range. Defaults to None.

    Returns:
    - None: Displays the plot.
    """
    fig = px.line(
        df_temp.melt(id_vars='datetime', value_vars=[y_column], var_name='component', value_name='value'),
        x='datetime',
        y='value',
        color='component',
        title=title,
        template='plotly_white',
        width=FIG_WIDTH, height=1.5*FIG_HEIGHT
    )
    if xlim:
        fig.update_xaxes(range=xlim)
    fig.update_layout(legend=dict(orientation='h'))
    fig.show()

In [10]:
temp = seasonal_decompose(df_taxi_orders.num_orders, model='additive', period=24)

df_temp = pd.DataFrame({
    'datetime': df_taxi_orders.index,
    'trend': temp.trend,
    'seasonal': temp.seasonal,
    'residual': temp.resid
})

generate_plot('trend', 'Trend over time')
generate_plot(
    'seasonal', 'Seasonal variation over time',
    [df_temp.datetime.iloc[0], df_temp.datetime.iloc[0] + pd.Timedelta(days=3)]
)
generate_plot('residual', 'Residuals over time')

## Модели ML

Создадим и обучим несколько моделей. Для начала разделим данные на `train` и `test` выборки. Создадим функцию для этого, чтобы сразу записать в датафрейм.

In [11]:
def split_data(df: pd.DataFrame, target_column: str, test_size: float, shuffle=False):
    """
    Split a DataFrame into training and testing datasets.

    This function accepts a DataFrame, the name of the target column, and the proportion of the data 
    to be included in the test split. It returns four DataFrames: the training features, the training target, 
    the testing features, and the testing target. The target datasets are DataFrames with a single column rather 
    than Series objects.

    Args:
    - df (pd.DataFrame):
        The DataFrame to split. This DataFrame should include both the features and the target.

    - target_column (str): 
        The name of the target column. This column will be separated from the features and returned 
        in the target DataFrames.

    - test_size (float):
        The proportion of the data to include in the test split. For example, if `test_size` is 0.3, 
        30% of the data will be used for the test split, and the rest will be used for the training split.
        
    - shuffle (boolean):
        A flag to shuffle (True) or not (False) the data when splitting into train and test.

    Returns
    - list of pd.DataFrame:
        A list containing four DataFrames: the training features, the training target, 
        the testing features, and the testing target.
    """
    df_train, df_test = train_test_split(
        df, test_size=test_size, random_state=RANDOM_SEED, shuffle=shuffle
    )
    
    ftr_train = df_train.drop(target_column, axis=1)
    tgt_train = df_train[[target_column]]
    ftr_test = df_test.drop(target_column, axis=1)
    tgt_test = df_test[[target_column]]
    
    return [ftr_train, tgt_train, ftr_test, tgt_test]

Сохраним выборки.

In [12]:
dct_splits = split_data(df_taxi_orders, 'num_orders', 0.1)

dct_splits = {
    'train': {'features': dct_splits[0], 'target': dct_splits[1]},
    'test': {'features': dct_splits[2], 'target': dct_splits[3]}
}

print(
    'Test to full sample size:', 
    round(100 * dct_splits['test']['target'].shape[0] / df_taxi_orders.shape[0], 2),
    '%'
)

Test to full sample size: 10.01 %


Зададим `study` для `optuna` - она сделает для нас оптимальные модели.

In [13]:
def optimize_regressors(ftr_train, tgt_train, n_trials: int):
    """
    Trains and optimizes regression models using Optuna.

    Args:
    - ftr_train, tgt_train: Training features and target.
    - n_trials (int): The number of trials for Optuna optimization.

    Returns:
    - An Optuna study object containing the optimal model and its parameters.
    """
    # Ensure target is 1-d vector
    tgt_train = tgt_train.values.ravel()

    def get_regressor(trial):
        regressors = {
            'LinearRegression': LinearRegression(),
            'RandomForest': RandomForestRegressor(
                max_depth=trial.suggest_int('max_depth', 1, 100),
                n_estimators=trial.suggest_int('n_estimators', 100, 1000),
                random_state=RANDOM_SEED
            ),
            'CatBoost': CatBoostRegressor(
                iterations=trial.suggest_int('iterations', 100, 1000),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
                logging_level='Silent',
                random_state=RANDOM_SEED
            ),
            'LGBM': LGBMRegressor(
                max_depth=trial.suggest_int('max_depth', 1, 50),
                n_estimators=trial.suggest_int('n_estimators', 100, 1000),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
                random_state=RANDOM_SEED
            )
        }
        regressor_name = trial.suggest_categorical('regressor', list(regressors.keys()))
        return regressors[regressor_name]

    def objective(trial):
        """
        Objective function for Optuna optimization. Computes the RMSE for a given regressor.

        Args:
        - trial (optuna.Trial): 
            A trial is a process of evaluating an objective function. 
            This object is passed to an objective function and provides interfaces to 
            suggest hyperparameters.

        Returns:
        - float:
            Root Mean Squared Error (RMSE) of the regressor's predictions.
        """
        regressor_obj = get_regressor(trial)
        predictions = cross_val_predict(regressor_obj, ftr_train, tgt_train, cv=3)
        return np.sqrt(mean_squared_error(tgt_train, predictions))

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=n_trials)
    
    return study

Возьмем результаты `study`.

In [17]:
study = optimize_regressors(
    dct_splits['train']['features'],
    dct_splits['train']['target'],
    n_trials=50
)

Посмотрим, как модели соотносятся друг с другом.

In [18]:
best_params = study.best_params
formatted_params = "\n".join([f"  {key}: {value}" for key, value in best_params.items()])
print(f"Best params:\n{formatted_params}")

# Plotting the optimization history
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.show()

# Plotting the slice plot
fig = optuna.visualization.plot_slice(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.update_xaxes(tickangle=-90)
fig.show()

Best params:
  max_depth: 4
  n_estimators: 606
  iterations: 927
  learning_rate: 0.07109791953112046
  regressor: CatBoost
